In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sn
import statistics as st
import random
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import cross_val_score
import scipy

#linear regression
from sklearn.linear_model import LinearRegression

#split dataset
from sklearn.model_selection import train_test_split 

# Lab: Cross-Validation and the Bootstrap

## 5.3.1 The Validation Set Approach

In [12]:
#Read dataset from R
Auto = sm.datasets.get_rdataset("Auto","ISLR").data

Firstly, we split the set of observations into two halves, by selecting a random subset of 196 observations out of the original 392 observations. We refer to these observations as the training set.

In [43]:
len(X_test)

196

In [51]:
X = Auto['horsepower'].values.reshape(-1,1)
y = Auto['mpg'].values.reshape(-1,1)

In [18]:
#transfer dataframe into list
Auto_list = Auto.values.tolist()
#split in training set
train = random.sample(Auto_list, 196)
train = pd.DataFrame(train)

In [52]:
#Split training set with n=196 and testing set with n=196
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

We then use the subset to fit a linear regression using only the observations corresponding to the training set.

In [148]:
#linear regression
lr = LinearRegression()  
lr.fit(X_train, y_train) #training the algorithm
y_pred = lr.predict(X_test)

We now estimate the response for all 392 observations, and we calculate the MSE of the 196 observations in the testing set. 

In [54]:
#MSE
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  

Mean Squared Error: 23.616617069669882


Therefore, the estimated test MSE for the linear regression fit is 23.617.

Next, we find the MSE for quadratic and cubic regression.

In [57]:
#quadratic regression
polynomial_features= PolynomialFeatures(degree=2)
X = polynomial_features.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

lr.fit(X_train, y_train) #training the algorithm
y_pred = lr.predict(X_test)

print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))

Mean Squared Error: 18.763031346897744


In [58]:
#cubic regression
polynomial_features= PolynomialFeatures(degree=3)
X = polynomial_features.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

lr.fit(X_train, y_train) #training the algorithm
y_pred = lr.predict(X_test)

print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))

Mean Squared Error: 18.194589356943503



The error rates are 18.76 and 18.19, respectively. If we choose a different training set instead, then we will obtain somewhat different errors on the validation set.

These results are consistent with our previous findings: a model that predicts mpg using a **quadratic function** of horsepower performs better than a model that involves only a linear function of horsepower, and there is little evidence in favor of a model that uses a cubic function of horsepower.

## 5.3.2 Leave-One-Out Cross-Validation (LOOCV)

In [72]:
X = Auto.horsepower; 
X = X[:,np.newaxis];
y = Auto.mpg
orders = np.arange(1,6)
mse_est = np.array([])
for index, order in enumerate(orders):
    poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)
    regress = LinearRegression()
    regress.fit(poly.fit_transform(X), y)
    print('Coefficients: Intercept, Beta(s)', regress.intercept_, regress.coef_)
    mse_est = np.append(mse_est, -np.mean(cross_val_score(regress, poly.fit_transform(X), y,
                                                                          cv=len(X), scoring='neg_mean_squared_error')))
print('\nThe estimated test MSEs = ', mse_est)

Coefficients: Intercept, Beta(s) 39.93586102117047 [-0.15784473]
Coefficients: Intercept, Beta(s) 56.90009970211297 [-0.46618963  0.00123054]
Coefficients: Intercept, Beta(s) 60.68478490666118 [-5.68850128e-01  2.07901126e-03 -2.14662591e-06]
Coefficients: Intercept, Beta(s) 47.567677230017296 [-7.66721350e-02 -4.34464058e-03  3.24511548e-05 -6.53036297e-08]
Coefficients: Intercept, Beta(s) -32.23055925051304 [ 3.70010897e+00 -7.14240776e-02  5.93108571e-04 -2.28107628e-06
  3.32955307e-09]

The estimated test MSEs =  [24.23151352 19.24821312 19.33498406 19.42443031 19.0332128 ]


We repeat this procedure for increasingly complex polynomial fits. To automate the process, we initiate a loop which iteratively fits polynomial regressions for polynomials of order i = 1 to i = 5, computes the associated cross-validation error, and stores it in the ith element of the vector cv.error. We begin by initializing the vector. From the result above can we see a sharp drop in the estimated test MSE between the linear and quadratic fits, but then no clear improvement from using higher-order polynomials.

## 5.3.3 k-Fold Cross-Validation

In [74]:
X = Auto.horsepower; 
X = X[:,np.newaxis];
y = Auto.mpg
orders = np.arange(1,10)
mse_est = np.array([])
for index, order in enumerate(orders):
    poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)
    regress = LinearRegression()
    regress.fit(poly.fit_transform(X), y)
    print('Coefficients: Intercept, Beta(s)', regress.intercept_, regress.coef_)
    mse_est = np.append(mse_est, -np.mean(cross_val_score(regress, poly.fit_transform(X), y,
                                                                          cv=10, scoring='neg_mean_squared_error')))
print('\nThe estimated test MSEs = ', mse_est)

Coefficients: Intercept, Beta(s) 39.93586102117047 [-0.15784473]
Coefficients: Intercept, Beta(s) 56.90009970211297 [-0.46618963  0.00123054]
Coefficients: Intercept, Beta(s) 60.68478490666118 [-5.68850128e-01  2.07901126e-03 -2.14662591e-06]
Coefficients: Intercept, Beta(s) 47.567677230017296 [-7.66721350e-02 -4.34464058e-03  3.24511548e-05 -6.53036297e-08]
Coefficients: Intercept, Beta(s) -32.23055925051304 [ 3.70010897e+00 -7.14240776e-02  5.93108571e-04 -2.28107628e-06
  3.32955307e-09]
Coefficients: Intercept, Beta(s) -162.14413961110918 [ 1.12381147e+01 -2.43636568e-01  2.58014076e-03 -1.45299453e-05
  4.17283090e-08 -4.80321240e-11]
Coefficients: Intercept, Beta(s) 14.488249578731345 [ 1.26465278e-03  4.69956315e-02 -1.39312752e-03  1.65893517e-05
 -9.84641105e-08  2.89696707e-10 -3.36931900e-13]
Coefficients: Intercept, Beta(s) 37.41710536512173 [ 5.07503727e-08  3.13402515e-06  9.50797255e-05 -4.46285569e-06
  6.36102185e-08 -4.12421843e-10  1.27136133e-12 -1.51546201e-15]
Coe

We still see little evidence that using cubic or higher-order polynomial terms leads to lower test error than simply using a quadratic fit.

## 5.3.4 The Bootstrap


One of the great advantages of the bootstrap approach is that it can be applied in almost all situations. No complicated mathematical calculations are required. 

### Estimating the Accuracy of a Statistic of Interest

In [96]:
#create function alpha.fn
def alpha_fn(data,index):
    X = data.X[index]
    Y = data.Y[index]
    return((st.variance(Y)-np.cov(X,Y)[0][1])/(st.variance(X)+st.variance(Y)-2*np.cov(X,Y)[0][1]))

In [87]:
#Read dataset from R
Portfolio = sm.datasets.get_rdataset("Portfolio","ISLR").data

In [99]:
a = list(range(100))

In [100]:
alpha_fn(Portfolio, a)

0.57583207459283

This function returns an estimate for α based on applying (5.7) to the observations indexed by the argument index. For instance, the following command tells Python to estimate α using all 100 observations.

The next command randomly select 100 observations from the range 1 to 100, with replacement. This is equivalent to constructing a new bootstrap data set and recomputing αˆ based on the new data set.

In [103]:
alpha_fn(Portfolio, np.random.choice(100,100, replace = True))

0.5184949611013724

We can implement a bootstrap analysis by performing this command many times, recording all of the corresponding estimates for α, and computing the resulting standard deviation.

In [104]:
#Boot function
def boot(data, statsfunc, num_bootstrap_samples=1000, sample_size=100):
    stat_samples = []
    for sample in range(num_bootstrap_samples):
        indices = np.random.choice(data.index, sample_size, replace=True)
        stat_samples.append(statsfunc(data, indices))
    se_estimate = scipy.std(stat_samples, axis=0)
    print('\nBootstrapped Std. Error(s) =', se_estimate)

In [109]:
boot(Portfolio, alpha_fn, 1000, 100) 


Bootstrapped Std. Error(s) = 0.09100260584187886


The final output shows that using the original data, the bootstrap estimate for SE(αˆ) is 0.091.

### Estimating the Accuracy of a Linear Regression Model

In [110]:
def boot_fn(data, index):
    X = data.horsepower[index].values
    X = sm.add_constant(X)
    y = data.mpg[index].values
    lm_fit = sm.OLS(y, X).fit()
    return lm_fit.params

boot_fn(Auto, Auto.index)

array([39.93586102, -0.15784473])

Or we can use another method to get the similar result. 

In [114]:
np.random.seed(1); 
indices = np.random.choice(Auto.index, len(Auto), replace=True)

boot_fn(Auto, indices) 

array([39.65847877, -0.15589835])

In [115]:
boot(Auto, boot_fn, num_bootstrap_samples=1000, sample_size=len(Auto))


Bootstrapped Std. Error(s) = [0.82521528 0.00713533]


This indicates that the bootstrap estimate for SE(β0) is 0.825, and that the bootstrap estimate for SE(β1) is 0.007. As discussed in Section 3.1.2, standard formulas can be used to compute the standard errors for the regression coefficients in a linear model. The summary of result is below:

In [116]:
smf.ols('mpg~horsepower', Auto).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.606
Model:                            OLS   Adj. R-squared:                  0.605
Method:                 Least Squares   F-statistic:                     599.7
Date:                Fri, 28 Feb 2020   Prob (F-statistic):           7.03e-81
Time:                        23:34:06   Log-Likelihood:                -1178.7
No. Observations:                 392   AIC:                             2361.
Df Residuals:                     390   BIC:                             2369.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.9359      0.717     55.660      0.000      38.525      41.347
horsepower    -0.1578      0.006    -24.489      0.000      -0.171      -0.145
==============================================================================
Omnibus:                       16.432   Durbin-Watson:                   0.920
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               17.305
Skew:                           0.492   Prob(JB):                     0.000175
Kurtosis:                       3.299   Cond. No.                         322.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Interestingly, these are somewhat different from the estimates obtained using the bootstrap. Although the formula for the standard errors do not rely on the linear model being correct, the estimate for σ2 does. There is a non-linear relationship in the data, and so the residuals from a linear fit will be inflated, and so will σˆ2. 

Below we compute the bootstrap standard error estimates and the standard linear regression estimates that result from fitting the quadratic model to the data. Since this model provides a good fit to the data (Figure 3.8), there is now a better correspondence between the bootstrap estimates and the standard estimates of SE(β0), SE(β1) and SE(β2).

In [117]:
def boot_fn(df, indices):
    X = np.column_stack([df.horsepower[indices].values, df.horsepower[indices].values**2])
    X = sm.add_constant(X)
    y = df.mpg[indices].values
    lm_fit = sm.OLS(y, X).fit()
    return lm_fit.params
boot(Auto, boot_fn, num_bootstrap_samples=1000, sample_size=len(Auto))


Bootstrapped Std. Error(s) = [2.09222532e+00 3.32006462e-02 1.19440327e-04]


In [118]:
smf.ols('mpg ~ horsepower + np.power(horsepower, 2)', Auto).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.686
Method:                 Least Squares   F-statistic:                     428.0
Date:                Fri, 28 Feb 2020   Prob (F-statistic):           5.40e-99
Time:                        23:38:36   Log-Likelihood:                -1133.2
No. Observations:                 392   AIC:                             2272.
Df Residuals:                     389   BIC:                             2284.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                  56.9001      1.800     31.604      0.000      53.360      60.440
horsepower                 -0.4662      0.031    -14.978      0.000      -0.527      -0.405
np.power(horsepower, 2)     0.0012      0.000     10.080      0.000       0.001       0.001
==============================================================================
Omnibus:                       16.158   Durbin-Watson:                   1.078
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.662
Skew:                           0.218   Prob(JB):                     2.20e-07
Kurtosis:                       4.299   Cond. No.                     1.29e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.29e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# #6

## (a)

In [123]:
smf.glm('default ~ income + balance', Default, family=sm.families.Binomial()).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                        Generalized Linear Model Regression Results                        
===========================================================================================
Dep. Variable:     ['default[No]', 'default[Yes]']   No. Observations:                10000
Model:                                         GLM   Df Residuals:                     9997
Model Family:                             Binomial   Df Model:                            2
Link Function:                               logit   Scale:                          1.0000
Method:                                       IRLS   Log-Likelihood:                -789.48
Date:                             Sat, 29 Feb 2020   Deviance:                       1579.0
Time:                                     11:27:37   Pearson chi2:                 6.95e+03
No. Iterations:                                  9                                         
Covariance Type:                         nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     11.5405      0.435     26.544      0.000      10.688      12.393
income     -2.081e-05   4.99e-06     -4.174      0.000   -3.06e-05    -1.1e-05
balance       -0.0056      0.000    -24.835      0.000      -0.006      -0.005
==============================================================================
"""

The estimated standard errors for the coefficients associated with income and balance is 4.99e-06 and 0, respectively.

## (b)

In [133]:
def boot_fn(default):
    model = smf.glm('default ~ income + balance', Default, family=sm.families.Binomial()).fit()
    coef_income = model.params[1]
    coef_balance = model.params[2]
    return [coef_income, coef_balance]

In [136]:
boot_fn(Default)

[-2.080897552898698e-05, -0.005647102950316488]

## (c)

In [139]:
#Boot function
def boot(data, statsfunc, num_bootstrap_samples=1000, sample_size=100):
    stat_samples = []
    for sample in range(num_bootstrap_samples):
        stat_samples.append(statsfunc(data))
    se_estimate = scipy.std(stat_samples, axis=0)
    print('\nBootstrapped Std. Error(s) =', se_estimate)

In [140]:
boot(Default, boot_fn, 1000, 100) 


Bootstrapped Std. Error(s) = [6.77626358e-20 3.20923843e-17]


## (d)

The result in part b is larger than that of part c.

# #7

In [141]:
#Read dataset from R
Weekly = sm.datasets.get_rdataset("Weekly","ISLR").data

## (a)

In [180]:
#transer Direction into dummy variable
Weekly['Direction_Up'] = (Weekly['Direction'] == 'Up').astype(int)

X = Weekly[['Lag1', 'Lag2']]
y = Weekly.Direction_Up

lr.fit(X,y)
print(lr.intercept_, lr.coef_, lr.predict(X).mean())  # accuracy

0.5547695090975024 [-0.00945186  0.01462384] 0.5555555555555556


## (b)

In [173]:
#remove first observation
lr.fit(X.iloc[1:],y.iloc[1:])

print(lr.intercept_, lr.coef_, lr.predict(X).mean())  # accuracy

0.5552615405384282 [-0.00937857  0.01476349] 0.5560797216067374


## (c)

In [203]:
print(lr.predict([X.iloc[0]]))

[0.56956525]


Because the first observation = 0.57 > 0.5, so we classified it as "Up". However, the true value is "Down".

## (d)

In [196]:
errors = np.zeros(len(X))
predicts = np.zeros(len(X))

for i in range(len(X)):
    lr.fit(X.drop([i]),y.drop([i]))
    if lr.predict([X.iloc[i]]) > 0.5:
        predicts[i] = 1
    else:
        predicts[i] = 0
    if predicts[i] != y[i]:
        errors[i] = 1

## (e)

In [197]:
errors.mean()

0.44811753902662993

The LOOCV estimate for the test error is 0.448, which means the model is not good at predicting.

# #9

In [205]:
#Read dataset from R
Boston = sm.datasets.get_rdataset("Boston","MASS").data

## (a)

In [209]:
#population mean of medv
mu = Boston.medv.mean()
mu

22.532806324110698

## (b)

In [210]:
SE_medv = Boston.medv.std()/np.sqrt(len(Boston))
SE_medv

0.4088611474975351

## (c)

In [223]:
#SE of mean using bootstrap
means = [Boston.medv.sample(n = len(Boston), replace=True).mean() for _ in range(1000)]
se = np.std(means)
se

0.39920199850137605

The estimated SE in part c is smaller than that in part b.

## (d)

In [224]:
print(mu-2*se, mu+2*se)

21.734402327107947 23.33121032111345


## (e)

In [225]:
mu_med = Boston.medv.median()
mu_med

21.2

## (f)

In [226]:
#SE of medain using bootstrap
median = [Boston.medv.sample(n = len(Boston), replace=True).median() for _ in range(1000)]
se_med = np.std(median)
se_med

0.38953810789189774

Compare to the standard error of mean, the SE of median is smaller.

## (g)

In [227]:
Boston.medv.quantile(0.1)

12.75

## (h)

In [228]:
#SE of quantile 0.1 using bootstrap
q = [Boston.medv.sample(n = len(Boston), replace=True).quantile(0.1) for _ in range(1000)]
se_qtl = np.std(q)
se_qtl

0.5015126493918174

The SE is 0.5, which is larger than that of part c and part f.